# Dreambooth Stable Diffusion - Winter 2022 Edition 

Updated 2022-03-08

This Colab is based on Shivam Shrirao's repository and has been modified to use dependencies from late 2022 but with diffusion from revision `fbdf0a17055ffa34679cb34d986fabc1296d0785` (2023-03-02).

If you prefer to use an alt layout, you can use [dbsd_dec_2022.ipynb](https://colab.research.google.com/github/yushan777/dbsd-dec-2022/blob/main/dbsd_dec_2022.ipynb)

___
https://github.com/yushan777/dbsd-xmas-edition

https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth


Join the Dreambooth Discord!
https://discord.gg/wNNs2JNF7G 
___
#### Instructions:
#### Run each cell in order.

In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15109 MiB, 15109 MiB


https://github.com/ShivamShrirao/diffusers/tree/main/examples/dreambooth

In [ ]:
#@title Install Requirements
# commit fbdf0a17055ffa34679cb34d986fabc1296d0785 2023-03-02

print("Installing ShivamShrirao/Diffusers...")
%pip install -q git+https://github.com/ShivamShrirao/diffusers.git@fbdf0a17055ffa34679cb34d986fabc1296d0785
%pip install torch==1.13.1+cu116 torchaudio==0.13.1+cu116 torchvision==0.14.1+cu116 --extra-index-url https://download.pytorch.org/whl/cu116 | grep -v 'already satisfied'
#%pip install -q torch==1.12.1+cu116 torchvision==0.13.1+cu116 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu116
%pip install omegaconf==2.3.0 # required for orig-diffusers conversion script
%pip install pytorch-lightning==1.8.5 # required for orig-diffusers conversion script
%pip install -U --pre triton==2.0.0.dev20221030 | grep -v 'already satisfied'
%pip install accelerate==0.12.0 transformers==4.26.0 ftfy==6.1.1 bitsandbytes==0.35.0 gradio natsort safetensors  | grep -v 'already satisfied'
%pip install xformers==0.0.16 | grep -v 'already satisfied'
#%pip install xformers==0.0.13 | grep -v 'already satisfied'

# get train_dreambooth.py and conversion scripts
!wget https://github.com/ShivamShrirao/diffusers/raw/fbdf0a17055ffa34679cb34d986fabc1296d0785/examples/dreambooth/train_dreambooth.py
!wget https://github.com/ShivamShrirao/diffusers/raw/fbdf0a17055ffa34679cb34d986fabc1296d0785/scripts/convert_diffusers_to_original_stable_diffusion.py
!wget https://github.com/ShivamShrirao/diffusers/raw/fbdf0a17055ffa34679cb34d986fabc1296d0785/scripts/convert_original_stable_diffusion_to_diffusers.py
!wget https://raw.githubusercontent.com/yushan777/dbsd-dec-2022/main/notification.mp3
# =====================================================================================================================
# remove instances of param 'keep_fp32_wrapper=True' from file 'train_dreambooth.py'
import fileinput
filename = 'train_dreambooth.py'
with fileinput.FileInput(filename, inplace=True, backup='~bak') as file:
    for line in file:
        print(line.replace(', keep_fp32_wrapper=True', ''), end='')


# =====================================================================================================================

print("Finished Installing Requirements.")







In [ ]:
#@title Settings Model Path
#@markdown Name/Path of the base model
MODEL_NAME = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}

#@markdown If you wish to convert a ckpt or safetensors file to diffusers go to cell [Download & Convert Ckpt, Safetensors To Diffusers](#scrollTo=K4curq5G4PBz)



In [ ]:
#@title Token & Class Word
token_word = "zwx" #@param {type:"string"}
class_word = "person" #@param {type:"string"}

In [ ]:
#@title Output Directory (where your trained model will be trained)
#@markdown If model weights should be saved directly in google drive \
save_to_gdrive = False #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive')

#@markdown It is recommended to only save to google drive AFTER training is complete otherwise you could run out of storage fast (esp. if you are saving at intervals as each weight takes around 4-5 GB + another 2GB when converting back to ckpt)\
OUTPUT_DIR = f"stable_diffusion_weights/{token_word}" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")
!mkdir -p $OUTPUT_DIR


### Training Parameters

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |


Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

remove `--train_text_encoder` flag to reduce memory usage further, degrades output quality.

In [ ]:
#@title 5. Download Regularization Images (faster than generating them prior to training.)
#@markdown We’ve created the following image sets
#@markdown - `man_euler` - provided by Niko Pueringer (Corridor Digital) - euler @ 40 steps, CFG 7.5
#@markdown - `man_unsplash` - pictures from various photographers
#@markdown - `person_ddim`
#@markdown - `woman_ddim` - provided by David Bielejeski - ddim @ 50 steps, CFG 10.0 <br />
#@markdown - `artstyle` - provided by Hackmans - ddim @ 50 steps, CFG 10.0 <br />
from IPython.display import clear_output

dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "artstyle"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mkdir -p regularization_images/{dataset}
!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* regularization_images/{dataset}

# remove temp folder now it is empty. 
!rm -rf Stable-Diffusion-Regularization-Images-{dataset}

clear_output()
print("✅ \033[92mRegularization Images downloaded.\033[0m")

In [ ]:
#@title Saved Concepts List File
# You can also add multiple concepts here. Try tweaking `--max_train_steps` accordingly.

concepts_list = [
    {
        "instance_prompt":      f"{token_word} {class_word}",
        "class_prompt":         f"{class_word}",
        "instance_data_dir":    f"/content/data/{token_word}",
        "class_data_dir":       f"/content/data/{class_word}"
    },
#     {
#         "instance_prompt":      "photo of ukj person",
#         "class_prompt":         "photo of a person",
#         "instance_data_dir":    "/content/data/ukj",
#         "class_data_dir":       "/content/data/person"
#     }
]

# `class_data_dir` contains regularization images
import json
import os
for c in concepts_list:
    os.makedirs(c["instance_data_dir"], exist_ok=True)

with open("concepts_list.json", "w") as f:
    json.dump(concepts_list, f, indent=4)

In [ ]:
#@markdown Upload your images by running this cell.

#@markdown OR

#@markdown You can use the file manager on the left panel to upload (drag and drop) to each `instance_data_dir` (it uploads faster)

import os
from google.colab import files
import shutil

for c in concepts_list:
    print(f"Uploading instance images for `{c['instance_prompt']}`")
    uploaded = files.upload()
    for filename in uploaded.keys():
        dst_path = os.path.join(c['instance_data_dir'], filename)
        shutil.move(filename, dst_path)

In [ ]:
#@title Training!

import time
from datetime import timedelta
start_time = time.time()


!accelerate launch train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --pretrained_vae_name_or_path="stabilityai/sd-vae-ft-mse" \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=800 \
  --save_interval=10000 \
  --save_sample_prompt="photo of zwx person" \
  --concepts_list="concepts_list.json"
time_taken = time.time() - start_time
# in HH:MM:SS
delta = str(timedelta(seconds=time_taken))

# Reduce the `--save_interval` to lower than `--max_train_steps` to save weights from intermediate steps.
# `--save_sample_prompt` can be same as `--instance_prompt` to generate intermediate samples (saved along with weights in samples directory).

print("Training Complete.  Time Taken: " + f'{delta}')

In [ ]:
#@markdown Specify the weights directory to use (leave blank for latest)
WEIGHTS_DIR = "" #@param {type:"string"}
if WEIGHTS_DIR == "":
    from natsort import natsorted
    from glob import glob
    import os
    WEIGHTS_DIR = natsorted(glob(OUTPUT_DIR + os.sep + "*"))[-1]
print(f"[*] WEIGHTS_DIR={WEIGHTS_DIR}")

In [ ]:
#@markdown Run to generate a grid of preview images from the last saved weights.
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

weights_folder = OUTPUT_DIR
folders = sorted([f for f in os.listdir(weights_folder) if f != "0"], key=lambda x: int(x))

row = len(folders)
col = len(os.listdir(os.path.join(weights_folder, folders[0], "samples")))
scale = 4
fig, axes = plt.subplots(row, col, figsize=(col*scale, row*scale), gridspec_kw={'hspace': 0, 'wspace': 0})

for i, folder in enumerate(folders):
    folder_path = os.path.join(weights_folder, folder)
    image_folder = os.path.join(folder_path, "samples")
    images = [f for f in os.listdir(image_folder)]
    for j, image in enumerate(images):
        if row == 1:
            currAxes = axes[j]
        else:
            currAxes = axes[i, j]
        if i == 0:
            currAxes.set_title(f"Image {j}")
        if j == 0:
            currAxes.text(-0.1, 0.5, folder, rotation=0, va='center', ha='center', transform=currAxes.transAxes)
        image_path = os.path.join(image_folder, image)
        img = mpimg.imread(image_path)
        currAxes.imshow(img, cmap='gray')
        currAxes.axis('off')
        
plt.tight_layout()
plt.savefig('grid.png', dpi=72)

## Convert weights to ckpt to use in web UIs like AUTOMATIC1111.

In [ ]:
#@markdown Run conversion.
ckpt_path = WEIGHTS_DIR + "/model.ckpt"

half_arg = ""
#@markdown  Whether to convert to fp16, takes half the space (2GB).
fp16 = True #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

## Inference

In [ ]:
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline, DDIMScheduler
from IPython.display import display

model_path = WEIGHTS_DIR             # If you want to use previously trained model saved in gdrive, replace this with the full path of model in gdrive

pipe = StableDiffusionPipeline.from_pretrained(model_path, safety_checker=None, torch_dtype=torch.float16).to("cuda")
pipe.scheduler = DDIMScheduler.from_config(pipe.scheduler.config)
pipe.enable_xformers_memory_efficient_attention()
g_cuda = None

In [ ]:
#@markdown Can set random seed here for reproducibility.
g_cuda = torch.Generator(device='cuda')
seed = 52362 #@param {type:"number"}
g_cuda.manual_seed(seed)

In [ ]:
#@title Run for generating images.

prompt = "photo of zwx dog in a bucket" #@param {type:"string"}
negative_prompt = "" #@param {type:"string"}
num_samples = 4 #@param {type:"number"}
guidance_scale = 7.5 #@param {type:"number"}
num_inference_steps = 24 #@param {type:"number"}
height = 512 #@param {type:"number"}
width = 512 #@param {type:"number"}

with autocast("cuda"), torch.inference_mode():
    images = pipe(
        prompt,
        height=height,
        width=width,
        negative_prompt=negative_prompt,
        num_images_per_prompt=num_samples,
        num_inference_steps=num_inference_steps,
        guidance_scale=guidance_scale,
        generator=g_cuda
    ).images

for img in images:
    display(img)

In [ ]:
#@title Download & Convert Ckpt, Safetensors To Diffusers 

%pip install omegaconf==2.3.0 # required for orig-diffusers conversion script
%pip install pytorch-lightning==1.8.5 # required for orig-diffusers conversion script

import os
from glob import glob
from natsort import natsorted

# =========================================================================================
# CHECKPOINTS / SAFETENSORS
# =========================================================================================

MODEL_NAME = ''

#@markdown Use this cell to directly download a checkpoint/safetensor model which will be converted. \
CKPT_SAFETENSOR_URL = 'https://civitai.com/api/download/models/6987?type=Model&format=SafeTensor' #@param {type:"string"}

if len(CKPT_SAFETENSOR_URL) == 0:
  CKPT_SAFETENSOR_URL = 'https://civitai.com/api/download/models/6987?type=Model&format=SafeTensor'

googlelink_prefix = 'https://drive.google.com/file/d/'
googlelink_suffix = '/view?usp=share_link'

download_dir = '/content/downloads'

if path.exists(download_dir)==False:
    os.mkdir(download_dir)

# check if CKPT_SAFETENSOR_url is a google drive share link
if CKPT_SAFETENSOR_URL.startswith(googlelink_prefix):
    share_id = CKPT_SAFETENSOR_URL
    print("is valid google share link")
    share_id = share_id.replace(googlelink_prefix, '')
    share_id = share_id.replace(googlelink_suffix, '')
    CKPT_SAFETENSOR_URL = f'https://drive.google.com/uc?id={share_id}'
    # gdown sucks on colab and fails for large files, we will have to use wget
    # see https://bcrf.biochem.wisc.edu/2021/02/05/download-google-drive-files-using-wget/
    !wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=$share_id' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=$share_id" --content-disposition -P "$download_dir" && rm -rf /tmp/cookies.txt    
else:
    #download model file into download dir
    !wget "$CKPT_SAFETENSOR_URL" --content-disposition -P "$download_dir"

# get the filename
search_pattern = f'{download_dir}/*'
file_list = natsorted(glob(f'{search_pattern}', recursive=False))

# if file_list is not empty...
if len(file_list)==0:
    print("Error: No files downloaded.")
else:  
    # get last file (should be the only file)
    file_path = file_list[-1]  

if file_path.endswith('.ckpt') or file_path.endswith('.safetensors'):    
    # filename only
    filename = os.path.basename(os.path.normpath(file_path))
    # save filename without extension
    filename_no_ext = os.path.splitext(filename)[0]
    #print(filename_no_ext)
    # if orig. filename ext. is safetensors then set parameter flag

    from_safetensors = ""
    if filename.endswith('safetensors'):
        from_safetensors = "--from_safetensors"

    # =========================================================================================
    # DIFFUSERS DIRECTORY
    # =========================================================================================
    DIFFUSERS_DIR = ""

    if len(DIFFUSERS_DIR)==0:
        DIFFUSERS_DIR = f'/content/diffusers-{filename_no_ext}'

    print("converting to diffusers... " + DIFFUSERS_DIR)


    # convert to diffusers 
    !python convert_original_stable_diffusion_to_diffusers.py --checkpoint_path "$file_path" --dump_path "$DIFFUSERS_DIR" $from_safetensors

    # set the model name/path
    MODEL_NAME=DIFFUSERS_DIR

    print("Model converted to diffusers : " + f'{DIFFUSERS_DIR}')
else:
    print("Error : File downloaded is not a ckpt or safetensors model file.")

#@markdown > Example URLs:<br>
#@markdown > `https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt` \
#@markdown >`https://civitai.com/api/download/models/6987?type=Model&format=PickleTensor` \
#@markdown >`https://civitai.com/api/download/models/6987?type=Model&format=SafeTensor` \
#@markdown >`https://drive.google.com/file/d/1JEZCyW36ziz9Fn482MUG8T0_2P4FGG-/view?usp=share_link` _(google drive share link)_ \
#@markdown >

#@markdown ___


In [ ]:
#@markdown Run Gradio UI for generating images.
import gradio as gr

def inference(prompt, negative_prompt, num_samples, height=512, width=512, num_inference_steps=50, guidance_scale=7.5):
    with torch.autocast("cuda"), torch.inference_mode():
        return pipe(
                prompt, height=int(height), width=int(width),
                negative_prompt=negative_prompt,
                num_images_per_prompt=int(num_samples),
                num_inference_steps=int(num_inference_steps), guidance_scale=guidance_scale,
                generator=g_cuda
            ).images

with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            prompt = gr.Textbox(label="Prompt", value="photo of zwx dog in a bucket")
            negative_prompt = gr.Textbox(label="Negative Prompt", value="")
            run = gr.Button(value="Generate")
            with gr.Row():
                num_samples = gr.Number(label="Number of Samples", value=4)
                guidance_scale = gr.Number(label="Guidance Scale", value=7.5)
            with gr.Row():
                height = gr.Number(label="Height", value=512)
                width = gr.Number(label="Width", value=512)
            num_inference_steps = gr.Slider(label="Steps", value=24)
        with gr.Column():
            gallery = gr.Gallery()

    run.click(inference, inputs=[prompt, negative_prompt, num_samples, height, width, num_inference_steps, guidance_scale], outputs=gallery)

demo.launch(debug=True)

In [ ]:
#@title (Optional) Delete diffuser and old weights and only keep the ckpt to free up drive space.

#@markdown [ ! ] Caution, Only execute if you are sure u want to delete the diffuser format weights and only use the ckpt.
import shutil
from glob import glob
import os
for f in glob(OUTPUT_DIR+os.sep+"*"):
    if f != WEIGHTS_DIR:
        shutil.rmtree(f)
        print("Deleted", f)
for f in glob(WEIGHTS_DIR+"/*"):
    if not f.endswith(".ckpt") or not f.endswith(".json"):
        try:
            shutil.rmtree(f)
        except NotADirectoryError:
            continue
        print("Deleted", f)

In [ ]:
#@title Free runtime memory
exit()